In [1]:
import sqlite3 as sql
import csv
import os
from pathlib import Path

import pandas as pd

In [3]:
REPO = (Path(_dh[-1]) / "..").resolve()
DATA = REPO / 'data'

# Map `chembl_targets` to `UniprotID`.

In [5]:
df = pd.read_csv(DATA / "human_kinases_and_chembl_targets.chembl_25.csv")
df = df[df.type == "SINGLE PROTEIN"].drop("type", axis=1)
df

,xName,Manning Name,HGNC Name,Kinase Name,Group,Family,SubFamily,UniprotID,chembl_targets
0,ABL1,ABL,ABL1,Tyrosine-protein kinase ABL1,TK,Abl,NaN,P00519,CHEMBL1862
5,ACK,ACK,TNK2,Activated CDC42 kinase 1,TK,Ack,NaN,Q07912,CHEMBL4599
6,ACTR2,ACTR2,ACVR2A,Activin receptor type-2A,TKL,STKR,STKR2,P27037,CHEMBL5616
7,ACTR2B,ACTR2B,ACVR2B,Activin receptor type-2B,TKL,STKR,STKR2,Q13705,CHEMBL5466
8,ADCK4,ADCK4,ADCK4,Uncharacterized aarF domain-containing protein...,Atypical,ABC1,ABC1-A,Q96D53,CHEMBL5753
...,...,...,...,...,...,...,...,...,...
819,TAF1L,TAF1L,TAF1L,Transcription initiation factor TFIID subunit ...,Atypical,TAF1,NaN,Q8IZX4,CHEMBL3108641
820,BCR,BCR,BCR,Breakpoint cluster region protein,Atypical,BCR,NaN,P11274,CHEMBL5146
822,Col4A3BP,Col4A3BP,COL4A3BP,Collagen type IV alpha-3-binding protein,Atypical,Col4A3BP,NaN,Q9Y5P4,CHEMBL3399913
823,BAZ1A,BAZ1A,BAZ1A,Bromodomain adjacent to zinc finger domain pro...,Atypical,BAZ,NaN,Q9NRL2,CHEMBL4105737


In [4]:
chembl_to_uniprot = {}
for row in df.itertuples():
    chembl_to_uniprot[row[-1]] = row[-2]

# Query local ChEMBL DB for speed

Get your local copy from http://doi.org/10.6019/CHEMBL.database.25. You need `chembl_25_sqlite.tar.gz` for this notebook. Extract it and point `DATABASE` below to the `chembl_25.db` file.

In [6]:
DATABASE = "chembl_25/chembl_25_sqlite/chembl_25.db"

In [7]:
conn = sql.connect(DATABASE, isolation_level=None)
db = conn.cursor()

Check which kind of assays can be found on human kinases.

In [8]:
q = f"""
SELECT standard_type, COUNT(standard_type) 
FROM activities
    LEFT JOIN assays ON assays.assay_id=activities.assay_id
    LEFT JOIN target_dictionary ON target_dictionary.tid=assays.tid
WHERE
    target_dictionary.chembl_id IN ({', '.join([f'"{x}"' for x in set(df.chembl_targets.tolist())])})
GROUP BY standard_type 
ORDER BY 2 DESC
"""
assay_types = pd.DataFrame.from_records(db.execute(q), columns=["Value", "Count"])
assay_types

,Value,Count
0,Inhibition,223964
1,IC50,213082
2,Ki,134114
3,Residual Activity,69836
4,Kd apparent,51695
...,...,...
114,TC50,1
115,fIC50,1
116,kmax,1
117,kmax/KS,1


There's a lot of information we are not using! `Inhibition` is as populated as `IC50`, but we don't know what kind of information this category contains.

Get all entries in the SQL db that:

- Correspond to IC50 or Ki measurements. Check `activities.standard_type` fields.
- assay_type = B (Binding)
- Relation = '='
- Target is part of the human kinome
- Confidence score is greather than zero (in practice, only 43 entries have score=0; the rest are either 8 or 9)

Then:
- Write Activity ID, Target ID, Publication Year and ID, Type of assay, measurement and units, canonical SMILES and Confidence scores to CSV
- Plot confidence value populations

Try:
- Cluster measurements by `doc_id` to homogeneize data (DOI is only available for ~100 000 entries, but doc_id is present in all rows would serve the same purpose, I guess!)

Note that querying for `docs.year` discards ~1000 entries because that field might not be available in those rows and the SELECT query gets dropped.

In [9]:
q = f"""
SELECT
    activities.activity_id, target_dictionary.chembl_id, 
    activities.standard_type, activities.standard_value, activities.standard_units, 
    compound_structures.canonical_smiles,
    component_sequences.sequence,
    assays.confidence_score, docs.doc_id, docs.year
FROM 
    activities
    LEFT JOIN assays ON assays.assay_id=activities.assay_id
    LEFT JOIN target_dictionary ON target_dictionary.tid=assays.tid
    LEFT JOIN compound_structures ON activities.molregno=compound_structures.molregno
    LEFT JOIN docs ON docs.doc_id=activities.doc_id
    LEFT JOIN target_components ON target_dictionary.tid=target_components.tid
    LEFT JOIN component_sequences ON target_components.component_id=component_sequences.component_id
WHERE
    target_dictionary.chembl_id IN ({', '.join([f'"{x}"' for x in set(df.chembl_targets.tolist())])})
AND
    activities.standard_relation="="
AND
    assays.assay_type="B"
AND
    activities.standard_type in ("IC50", "Ki")
AND
    assays.confidence_score > 0
"""
result = db.execute(q)

In [10]:
with open('activities.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["activities.activity_id", "target_dictionary.chembl_id", 
                     "activities.standard_type", "activities.standard_value", "activities.standard_units", 
                     "compound_structures.canonical_smiles", 
                     "component_sequences.sequence",
                     "assays.confidence_score", "docs.doc_id", "docs.year", 
                     "UniprotID"])
    for row in result:  # iterate over the query result to avoid loading everything in memory
        writer.writerow(row + (chembl_to_uniprot[row[1]],))

In [11]:
activities = pd.read_csv("activities.csv")
activities

,activities.activity_id,target_dictionary.chembl_id,activities.standard_type,activities.standard_value,activities.standard_units,compound_structures.canonical_smiles,component_sequences.sequence,assays.confidence_score,docs.doc_id,docs.year,UniprotID
0,32260,CHEMBL203,IC50,41.00,nM,Cc1cc(C)c(\C=C\2/C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4...,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,8,4959,2002.0,P00533
1,32262,CHEMBL279,IC50,16500.00,nM,Cc1cc(C)c(\C=C\2/C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4...,MQSKVLLAVALWLCVETRAASVGLPSVSLDLPRLSIQKDILTIKAN...,8,4959,2002.0,P35968
2,32267,CHEMBL203,IC50,170.00,nM,Cc1cc([nH]c1\C=C\2/C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)...,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,8,4959,2002.0,P00533
3,32330,CHEMBL258,IC50,140.00,nM,Nc1ncnc2c1c(cn2C3CCCC3)c4cccc(Oc5ccccc5)c4,MGCGCSSHPEDDWMENIDVCENCHYPIVPLDGKGTLLIRNGSEVRD...,9,3891,2000.0,P06239
4,32331,CHEMBL258,IC50,1180.00,nM,Nc1ncnc2c1c(cn2C3CCCC3)c4cccc(Oc5ccccc5)c4,MGCGCSSHPEDDWMENIDVCENCHYPIVPLDGKGTLLIRNGSEVRD...,9,3891,2000.0,P06239
...,...,...,...,...,...,...,...,...,...,...,...
174752,18243937,CHEMBL5888,Ki,40.00,nM,CC(C)(O)C#Cc1ccc2OCC(F)(F)c3sc(nc3c2c1)C(=O)N,MAVMEMACPGAPGSAVGQQKELPKAKEKTPPLGKKQSSVYKLEAVE...,9,105910,2017.0,Q99558
174753,18243938,CHEMBL5888,Ki,20.00,nM,CC(C)(O)C#Cc1ccc2C3CC(C3)n4cc(nc4c2c1)C(=O)N,MAVMEMACPGAPGSAVGQQKELPKAKEKTPPLGKKQSSVYKLEAVE...,9,105910,2017.0,Q99558
174754,18243939,CHEMBL5888,Ki,2.90,nM,CC(C)(O)C#Cc1cc2c(cc1F)C3CC(C3)n4cc(nc24)C(=O)N,MAVMEMACPGAPGSAVGQQKELPKAKEKTPPLGKKQSSVYKLEAVE...,9,105910,2017.0,Q99558
174755,18243940,CHEMBL5888,Ki,0.40,nM,Cc1onc(c1)[C@](C)(O)C#Cc2cc3c(cc2F)C4CC(C4)n5c...,MAVMEMACPGAPGSAVGQQKELPKAKEKTPPLGKKQSSVYKLEAVE...,9,105910,2017.0,Q99558


In [12]:
counts = activities["assays.confidence_score"].value_counts()
counts

9    96916
8    77841
Name: assays.confidence_score, dtype: int64

In [13]:
total_available_measurements = assay_types[assay_types['Value']=='Ki']['Count'].values[0] + assay_types[assay_types['Value']=='IC50']['Count'].values[0]
print(f'Recovered {(100*(counts[9] + counts[8])/total_available_measurements):.2f}% ({counts[9] + counts[8]} out of {total_available_measurements})')

Recovered 50.33% (174757 out of 347196)
